A short notebook demonstrating an out of memory error using RDKit fingerprints to build a RandomForest model

In [49]:
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
from tqdm.notebook import tqdm
from cuml import RandomForestRegressor 
import cudf

Read a SMILES file into a dataframe

In [50]:
df = pd.read_csv('A2a.smi',sep=" ",names="SMILES Name IC50".split())

In [51]:
df

,SMILES,Name,IC50
0,CNCC1CC2c3ccccc3Cc3ccccc3N2O1,CHEMBL81485,6.360
1,CN(C)CC1CC2c3ccccc3Cc3ccccc3N2O1,CHEMBL83658,6.430
2,CN1CCN2c3ccccc3Cc3ccccc3C2C1,CHEMBL6437,7.525
3,CN(C)CC1CC2c3ccccc3Cc3ccc(Cl)cc3N2O1,CHEMBL315772,6.320
4,CC(N)Cc1c[nH]c2ccc3c(c12)CCCO3,CHEMBL133455,4.720
...,...,...,...
198,CCN1CCCC1CNC(=O)c1cc(S(N)(=O)=O)ccc1OC,CHEMBL26,5.700
199,O=C(c1ccc(OCCN2CCCCC2)cc1)c1c(-c2ccc(O)cc2)sc2...,CHEMBL81,5.780
200,N=C(Nc1ccc2c(c1)CCN2C1CCNCC1)c1cccs1,CHEMBL2203713,4.620
201,Cc1ccc(OCC(O)C(C)NC(C)C)c2c1CCC2,CHEMBL513389,9.300


A function to generate RDKit fingerprints from SMILES

In [52]:
def fp_list_from_smiles_list(smiles_list,n_bits=2048):
    fp_list = []
    for smiles in tqdm(smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            fp_list.append(fp_as_array(mol,n_bits))
        else:
            fp_list.append(None)
    return fp_list

def fp_as_array(mol,n_bits=2048):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
    arr = np.zeros((1,), np.int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

Generate the fingerprints

In [53]:
fp_list = fp_list_from_smiles_list(df.SMILES)

A function to convert a numpy array into a cudf

In [37]:
def np2cudf(arr):
    # convert numpy array to cudf dataframe
    df = pd.DataFrame({'fea%d'%i:arr[:,i] for i in range(arr.shape[1])})
    pdf = cudf.DataFrame()
    for c,column in enumerate(df):
        pdf[str(c)] = df[column]
    return pdf

Put X and y into cudf

In [54]:
X = np2cudf(np.array(fp_list,dtype=np.float32))
y = cudf.Series(df.IC50)

Try to fit the model and get an out of memory error

In [55]:
rf.fit(X,y)

MemoryError: std::bad_alloc: CUDA error at: /conda/conda-bld/librmm_1591196551527/work/include/rmm/mr/device/cuda_memory_resource.hpp66: cudaErrorMemoryAllocation out of memory